## Simple ReAct Agent for Calculator
The following code builds a Calculator.
Tools are defined for each calculation function.
Prompts provide a natural language question.
A ReAct agent runs stepwise to generate the right outcome by using the right tools. 

In [1]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI
from llama_index.core.tools import FunctionTool

In [2]:
def add(a: float, b: float) -> float:
  """Add two numbers and returns the result"""
  return a + b
add_tool = FunctionTool.from_defaults(fn=add)

def multiply(a: float, b: float) -> float:
  """Multiply two numbers and returns the result"""
  return a * b
multiply_tool = FunctionTool.from_defaults(fn=multiply)

def subtract(a: float, b: float) -> float:
  """Subtract two numbers and returns the result"""
  return a - b
subtract_tool = FunctionTool.from_defaults(fn=subtract)

def divide(a: float, b: float) -> float:
  """Divide two numbers and returns the result. Handles division by zero error"""
  if b == 0:
    raise ZeroDivisionError("Division by zero is not allowed")
  return a / b
divide_tool = FunctionTool.from_defaults(fn=divide)

# Scientific functions
def power(a: float, b: float) -> float:
  """Raise a number to a power and returns the result"""
  return a ** b
power_tool = FunctionTool.from_defaults(fn=power)

def factorial(n: int) -> int:
  """Calculate the factorial of a non-negative integer and returns the result"""
  if n < 0:
    raise ValueError("Factorial is not defined for negative numbers")
  if n == 0:
    return 1
  else:
    return n * factorial(n-1)
factorial_tool = FunctionTool.from_defaults(fn=factorial)

def sine(x: float) -> float:
  """Calculate the sine of an angle in radians and returns the result. Uses the math library"""
  import math
  return math.sin(x)
sine_tool = FunctionTool.from_defaults(fn=sine)

def cosine(x: float) -> float:
  """Calculate the cosine of an angle in radians and returns the result. Uses the math library"""
  import math
  return math.cos(x)
cosine_tool = FunctionTool.from_defaults(fn=cosine)

def tangent(x: float) -> float:
  """Calculate the tangent of an angle in radians and returns the result. Handles division by zero error"""
  if cosine(x) == 0:
    raise ZeroDivisionError("Tangent is not defined for angles where cosine is zero")
  return sine(x) / cosine(x)
tangent_tool = FunctionTool.from_defaults(fn=tangent)

In [3]:
# list of tools

tools = [multiply_tool, add_tool, subtract_tool, divide_tool, power_tool, factorial_tool, sine_tool, cosine_tool, tangent_tool]

In [6]:
from dotenv import load_dotenv
import os 

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = api_key

llm = OpenAI(model="gpt-4o-mini")
agent = ReActAgent.from_tools(tools, llm=llm, verbose=True)

In [7]:
response = agent.chat("What is 20+(2*4)? Calculate step by step ")

> Running step ed6f814e-33f3-4d28-a9f0-4434e029dc99. Step input: What is 20+(2*4)? Calculate step by step 
Thought: The current language of the user is: English. I need to perform the calculation step by step using the appropriate tools.
Action: multiply
Action Input: {'a': 2, 'b': 4}
Observation: 8
> Running step 330dc081-dfc6-453d-a631-ef8c7abd4fd5. Step input: None
Thought: Now that I have the result of the multiplication (2 * 4 = 8), I can proceed to add this result to 20.
Action: add
Action Input: {'a': 20, 'b': 8}
Observation: 28
> Running step c31049ea-1eee-467c-bea2-56eb68acb81d. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: The result of the calculation 20 + (2 * 4) is 28.


In [8]:
response = agent.query("Calculate cos(30) + tan(45)")

> Running step 05bd8c21-dd69-4935-8c61-42c4f24b176e. Step input: Calculate cos(30) + tan(45)
Thought: The current language of the user is: English. I need to use tools to calculate the cosine of 30 degrees and the tangent of 45 degrees, and then add the results together.
Action: cosine
Action Input: {'x': 0.5235987755982988}
Observation: 0.8660254037844387
> Running step 77c8cbcd-910e-42fe-af10-168918ddcc83. Step input: None
Thought: I have the result for cos(30), which is approximately 0.8660254037844387. Now I will calculate tan(45).
Action: tangent
Action Input: {'x': 0.7853981633974483}
Observation: 0.9999999999999999
> Running step 966ef358-9a91-4aef-bedf-ad6d8c892454. Step input: None
Thought: I have the result for tan(45), which is approximately 0.9999999999999999. Now I will add the two results together: cos(30) + tan(45).
Action: add
Action Input: {'a': 0.8660254037844387, 'b': 0.9999999999999999}
Observation: 1.8660254037844386
> Running step d1f7761a-e357-4461-a3de-33ff8dc21